# RSA

In [1]:
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
subj = 'subj001'
subj_date = 'subj001_10062023';
data_path = '../data'
rdm_path = f'{data_path}/interim/PairwiseDecoding/{subj}'
annotation_file = f'{data_path}/raw/annotations/annotations.csv'

## Load RDM

The RDMs are saved separately for each iteration (of 20). Load the RDMs and average them. Then plot the average RDM across time. This visualization is a check of the data quality. 

In [4]:
files = sorted(glob(f'{rdm_path}/*.csv'))
rdm = []
for f in files:
    rdm.append(pd.read_csv(f))
rdm = pd.concat(rdm)
average_rdm = rdm.groupby(['v1', 'v2', 'time']).mean()

In [5]:
average_rdm.head()

c1   c2  accuracy  perm
v1                 v2                 time                            
-YwZOeyAQC8_15.mp4 1AIVH5cEWrI_35.mp4 -0.199  0.0  1.0       0.5  10.0
                                      -0.194  0.0  1.0       0.5  10.0
                                      -0.189  0.0  1.0       0.5  10.0
                                      -0.184  0.0  1.0       0.5  10.0
                                      -0.179  0.0  1.0       0.5  10.0

In [ ]:
plotting_data = average_rdm.groupby('time').mean().accuracy.to_numpy()
ymin, ymax = plotting_data.min()-.01, plotting_data.max()+.01

In [ ]:
_, ax = plt.subplots()
ax.plot(time, plotting_data)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Decoding accuracy')
ax.vlines(x=[0, 0.5], ymin=ymin, ymax=ymax,
          colors='gray', linestyles='dashed', zorder=0)
ax.hlines(y=.5, xmin=time.min(), xmax=time.max(),
          colors='gray', linestyles='solid', zorder=0)
ax.set_xlim([time.min(), time.max()])
ax.set_ylim([ymin, ymax])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

## Feature RDMs

Load the feature annotations and make the RDMs. Care needs to be paid to ensure that that the order of the vidoes across the two RDMs is consistent. 

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
features_to_exclude = ['dominance', 'cooperation', 'intimacy']

In [ ]:
df = pd.read_csv(annotation_file).set_index('video_name')
df = df.drop(columns=features_to_exclude)
df.head()

In [ ]:
feature_rdms = {}
for col in df.columns:
    arr = np.expand_dims(df[col].to_numpy(), axis=1)
    feature_rdms[col] = pdist(arr, metric='euclidean')

## Correlate Features and EEG

In [ ]:
from src.tools import corr

In [ ]:
rsa = {}
for key, val in feature_rdms.items():
    time_corr = np.zeros(average_upper_rdm.shape[-1])
    for t in range(average_upper_rdm.shape[-1]):
        time_corr[t] = corr(val, average_upper_rdm[:, t])
    rsa[key] = time_corr

In [ ]:
for key, val in rsa.items():
    plt.plot(val, label=key)
plt.legend()

In [ ]:
plt.plt